In [6]:
import random
import glob
import numpy as np
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import os
import time
import sys
import sklearn
from tensorflow import keras
import tensorflow as tf
%matplotlib inline

In [24]:
# 设置参数
data_dir = "GHIM-20"
train_txt = os.path.join(data_dir, "list_train_shuffle.txt")
val_txt = os.path.join(data_dir, "list_val_shuffle.txt")
train_dir = data_dir
val_dir = data_dir

AUTOTUNE = tf.data.experimental.AUTOTUNE
height = 224
width = 224
channels = 3
BATCH_SIZE = 32
num_classes = 20
base_lr = 1e-2
gamma = 0.1
momentum = 0.9
epochs = 200

In [3]:
def tf_load_and_prase_data_v2(train_txt, val_txt = None):
    """
        读入数据集，并返回可使用的tf对象
        
        input:
            train_txt: 训练集数据
            val_txt：验证集数据
        
        return：
            train_ds, val_ds
    """
    def read_lines_from_txt(txt):
        lines = []
        with open(txt, 'r') as f:
            lines = f.readlines()
        return lines
    
    def split_dataset(dataset, prop = 0.05):
        train_data = []
        val_data = []
        for data in dataset:
            if random.random() > prop:
                train_data.append(data)
            else:
                val_data.append(data)
        return train_data, val_data
    
    def tf_read_and_prase_data(dataset):
        """
            input:
                dataset: [lines, lines, lines, ...]
            
            output:
                ds
        """
        def preprocess_image(image):
            image = tf.image.decode_jpeg(image, channels = channels)
            image = tf.image.resize(image, [height, width])
            image /= 255.0  # normalize to [0,1] range
            return image

        def load_and_preprocess_image(path):
            image = tf.io.read_file(path)
            return preprocess_image(image)
        
        # 分割数据
        images_path = []
        labels = []
        for data in dataset:
            if len(data.strip().split(" ")[1:]) != 0:
                images_path.append(data.strip().split(" ")[0])
                labels.append(int(data.strip().split(" ")[1]))

        # 读取数据
        ds = tf.data.Dataset.from_tensor_slices((images_path, labels))
        
        # 解析数据
        return ds
        
    
    # 输入判断
    if not os.path.exists(train_txt):
        raise ValueError("can not find the file:" + train_txt)             
    if val_txt is not None and not os.path.exists(val_txt):
        raise ValueError("can not find the file:" + val_txt)
    
    
    # 区分 && 打乱数据集
    if val_txt is not None:
        train_path_label = read_lines_from_txt(train_txt)
        val_path_label = read_lines_from_txt(val_txt)
    else:
        train_data = read_lines_from_txt(train_txt)
        train_path_label, val_path_label = split_dataset(train_data)
    
    num_train = len(train_path_label)
    num_valid = len(val_path_label)
    print("numbers of train dataset:{}, numbers of val dataset:{}".format(num_train, num_valid))
    
    # 读入 && 解析数据
    train_ds = tf_read_and_prase_data(train_path_label)
    val_ds = tf_read_and_prase_data(val_path_label)
    
    return train_ds, num_train, val_ds, num_valid

In [26]:
def tf_shuffle_data(dataset, buffer_size, batch_size):
    def normalize(x, mean=tf.constant([0.485, 0.456, 0.406]), std= tf.constant([0.229, 0.224, 0.225])):
        # 标准化
        # x: [224, 224, 3]
        # mean: [224, 224, 3], std: [3]
        x = (x - mean)/std
        return x
    
    def preprocess(x,y, data_argumentation = False):
        # x: 图片的路径，y：图片的数字编码
        x = tf.io.read_file(x)
        x = tf.image.decode_jpeg(x, channels=3) # RGBA
        x = tf.image.resize(x, [height, width])

        if data_argumentation:
            x = tf.image.random_flip_left_right(x)
            x = tf.image.random_flip_up_down(x)
            x = tf.image.random_crop(x, [224,224,3])

            # x: [0,255]=> -1~1
            x = tf.cast(x, dtype=tf.float32) / 255.
            x = normalize(x)
        else:
            x /= 255.0
        y = tf.convert_to_tensor(y)
        y = tf.one_hot(y, depth=num_classes)

        return x, y
    
    ds = dataset.shuffle(buffer_size=buffer_size)
    ds = ds.repeat()
    ds = ds.map(preprocess).batch(batch_size)
    # 当模型在训练的时候，`prefetch` 使数据集在后台取得 batch。
    ds = ds.prefetch(buffer_size = tf.data.experimental.AUTOTUNE)
    return ds

In [27]:
# 转换成tf
txt = "./list_train.txt"
train_origin_ds, num_train, val_origin_ds, num_valid = tf_load_and_prase_data_v2(txt)

# 打乱数据集
train_ds = tf_shuffle_data(train_origin_ds, num_train, BATCH_SIZE)
val_ds = tf_shuffle_data(val_origin_ds, num_valid, BATCH_SIZE)

numbers of train dataset:1091, numbers of val dataset:68


In [28]:
# 创建模型
model = keras.models.Sequential([
    keras.layers.Conv2D(64, (3,3), padding = 'same', activation = 'relu', strides = 2, input_shape = (height, width, channels)),
    keras.layers.Conv2D(64, (3,3), padding = 'same', activation = 'relu', strides = 2),
    keras.layers.Conv2D(128, (3,3), padding = 'same', activation = 'relu', strides = 2),
    keras.layers.Conv2D(128, (3,3), padding = 'same', activation = 'relu', strides = 2),
    keras.layers.Conv2D(256, (3,3), padding = 'same', activation = 'relu', strides = 2),
    keras.layers.AveragePooling2D((7,7)),
    keras.layers.Flatten(),
    keras.layers.Dense(num_classes, activation = 'softmax')
])

In [29]:
# 创建优化器等 
model.compile(loss = "categorical_crossentropy",
              opentimizer = 'adam', metrics = ['accuracy'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_15 (Conv2D)           (None, 112, 112, 64)      1792      
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 56, 56, 64)        36928     
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 28, 28, 128)       73856     
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 14, 14, 128)       147584    
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 7, 7, 256)         295168    
_________________________________________________________________
average_pooling2d_3 (Average (None, 1, 1, 256)         0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 256)              

In [30]:
# 训练
history = model.fit_generator(train_ds,
                              steps_per_epoch= num_train // BATCH_SIZE,
                              epochs= epochs,
                              validation_data=val_ds,
                              validation_steps= num_valid // BATCH_SIZE
                             )

Epoch 1/200
34/34 [==============================] - 12s 353ms/step - loss: 1.9653 - accuracy: 0.2408 - val_loss: 1.6452 - val_accuracy: 0.3906
Epoch 2/200
34/34 [==============================] - 11s 323ms/step - loss: 1.3028 - accuracy: 0.4366 - val_loss: 1.0696 - val_accuracy: 0.6094
Epoch 3/200
34/34 [==============================] - 11s 326ms/step - loss: 1.1555 - accuracy: 0.5487 - val_loss: 1.0288 - val_accuracy: 0.5938
Epoch 4/200
34/34 [==============================] - 11s 318ms/step - loss: 0.9713 - accuracy: 0.6526 - val_loss: 0.8543 - val_accuracy: 0.6719
Epoch 5/200
34/34 [==============================] - 11s 332ms/step - loss: 0.8519 - accuracy: 0.7059 - val_loss: 0.5783 - val_accuracy: 0.8281
Epoch 6/200
34/34 [==============================] - 11s 325ms/step - loss: 0.7971 - accuracy: 0.7417 - val_loss: 0.7278 - val_accuracy: 0.7344
Epoch 7/200
34/34 [==============================] - 11s 322ms/step - loss: 0.7377 - accuracy: 0.7794 - val_loss: 0.8035 - val_accuracy:

34/34 [==============================] - 12s 348ms/step - loss: 0.1010 - accuracy: 0.9715 - val_loss: 0.2269 - val_accuracy: 0.9531
Epoch 58/200
34/34 [==============================] - 11s 338ms/step - loss: 0.0894 - accuracy: 0.9733 - val_loss: 0.1139 - val_accuracy: 0.9688
Epoch 59/200
34/34 [==============================] - 12s 353ms/step - loss: 0.1203 - accuracy: 0.9660 - val_loss: 0.1774 - val_accuracy: 0.9688
Epoch 60/200
34/34 [==============================] - 12s 346ms/step - loss: 0.1068 - accuracy: 0.9779 - val_loss: 0.2502 - val_accuracy: 0.9375
Epoch 61/200
34/34 [==============================] - 12s 346ms/step - loss: 0.1108 - accuracy: 0.9770 - val_loss: 0.2265 - val_accuracy: 0.9375
Epoch 62/200
34/34 [==============================] - 12s 356ms/step - loss: 0.0653 - accuracy: 0.9807 - val_loss: 0.1589 - val_accuracy: 0.9531
Epoch 63/200
34/34 [==============================] - 12s 358ms/step - loss: 0.1289 - accuracy: 0.9697 - val_loss: 0.1477 - val_accuracy: 0.968

34/34 [==============================] - 11s 312ms/step - loss: 0.0544 - accuracy: 0.9770 - val_loss: 0.3257 - val_accuracy: 0.9375
Epoch 114/200
34/34 [==============================] - 11s 334ms/step - loss: 0.0755 - accuracy: 0.9807 - val_loss: 0.2115 - val_accuracy: 0.9531
Epoch 115/200
34/34 [==============================] - 11s 318ms/step - loss: 0.0244 - accuracy: 0.9881 - val_loss: 0.2496 - val_accuracy: 0.9531
Epoch 116/200
34/34 [==============================] - 11s 335ms/step - loss: 0.0412 - accuracy: 0.9835 - val_loss: 0.2180 - val_accuracy: 0.9375
Epoch 117/200
34/34 [==============================] - 11s 323ms/step - loss: 0.0247 - accuracy: 0.9908 - val_loss: 0.4841 - val_accuracy: 0.9375
Epoch 118/200
34/34 [==============================] - 11s 318ms/step - loss: 0.0681 - accuracy: 0.9825 - val_loss: 0.1743 - val_accuracy: 0.9531
Epoch 119/200
34/34 [==============================] - 11s 333ms/step - loss: 0.0708 - accuracy: 0.9798 - val_loss: 0.2560 - val_accuracy:

34/34 [==============================] - 12s 340ms/step - loss: 0.0137 - accuracy: 0.9917 - val_loss: 0.2194 - val_accuracy: 0.9688
Epoch 170/200
34/34 [==============================] - 11s 320ms/step - loss: 0.0582 - accuracy: 0.9789 - val_loss: 0.3426 - val_accuracy: 0.9375
Epoch 171/200
34/34 [==============================] - 11s 335ms/step - loss: 0.0362 - accuracy: 0.9881 - val_loss: 3.4439 - val_accuracy: 0.6406
Epoch 172/200
34/34 [==============================] - 11s 328ms/step - loss: 0.0897 - accuracy: 0.9844 - val_loss: 0.2763 - val_accuracy: 0.9375
Epoch 173/200
34/34 [==============================] - 11s 322ms/step - loss: 0.0416 - accuracy: 0.9816 - val_loss: 0.4752 - val_accuracy: 0.9375
Epoch 174/200
34/34 [==============================] - 11s 329ms/step - loss: 0.0164 - accuracy: 0.9945 - val_loss: 0.3291 - val_accuracy: 0.9531
Epoch 175/200
34/34 [==============================] - 11s 324ms/step - loss: 0.0421 - accuracy: 0.9825 - val_loss: 0.4417 - val_accuracy: